In [1]:
def randomprime(i):
    return random_prime(2**i-1,True,2**(i-1))

In [2]:
l = 2048

q = randomprime(l)
p = randomprime(l)

n = p * q
phi = (p-1)*(q-1)

e = ZZ.random_element(phi)
R = IntegerModRing(n)

while gcd(e, phi) != 1:
    e = ZZ.random_element(phi)

bezout = xgcd(e,phi)
d = Integer(mod(bezout[1],phi));

In [3]:
# Public key
(n,e)

# Private key
(p,q,d)

(16415518432552902498940174490510716133221179450378206747877675400852795624525733273135156356490090196768577372169695269483314868244314770443726742841770511803706686413625260920348783215874668929638027349476785809919266997594654117324707416399560742325776294031509985443561811546956796369041709350494541064859169742685392141226424527176298061040976240924282308001407261986972614955729867822723336435427052289144940987039564858434655119315754274762069404839497777161564323845471676985575389132640494306213464465705452610016467302380165123545298341173047598853119751480800786160429559710512747537979979480322077213622263,
 1749012887783200156831777816098038055736963257651270886708405574454736313509262742347744649625951932251181780603714187439467865423360922849429088793833296387794931257582924236354517932308109103355557727729484777865972736464896647936826221051955982775917387845381455523017202579808643412844871412111221557562365243120018508353076585975770097818133430610327553604847029558774207246

In [23]:
import hashlib
import binascii

from random import SystemRandom

nBits = 1024
k0BitsInt = 256
k0BitsFill = '0256b'
encoding = 'utf-8'

oracle1 = hashlib.sha256()
oracle2 = hashlib.sha256()

def CharsToBinary(msg):
    bits = bin(int(binascii.hexlify(msg), 16))
    return bits

def BinaryToChars(bits):
    n = int(bits, 2)
    return binascii.hexlify('%x' % n)

def paddingOAEP(msg):

    #oracle1 = hashlib.sha256()
    #oracle2 = hashlib.sha256()
    
    randBitStr = format(SystemRandom().getrandbits(k0BitsInt), k0BitsFill)
    binMsg = CharsToBinary(msg)
    
    if len(binMsg) <= (nBits-k0BitsInt):
        k1Bits = nBits - k0BitsInt - len(binMsg) 
        zeroPaddedMsg = binMsg + ('0'*k1Bits)

    oracle1.update(randBitStr.encode(encoding))
    x = format(int(zeroPaddedMsg, 2) ^^ int(oracle1.hexdigest(), 16), '0768b')
    oracle2.update(x.encode(encoding))
    y = format(int(randBitStr, 2) ^^ int(oracle2.hexdigest(), 16), k0BitsFill)

    return x+y

def unpad(msg):

    #oracle1 = hashlib.sha256()
    #oracle2 = hashlib.sha256()

    x = msg[0:768]
    y = msg[768:]

    oracle2.update(x.encode(encoding))
    r = format(int(y,2) ^^ int(oracle2.hexdigest(), 16), k0BitsFill)

    oracle1.update(r.encode(encoding))
    msgWith0s = format( int(x,2) ^^ int(oracle1.hexdigest(), 16), '0768b')

    return BinaryToChars(msgWith0s)

In [24]:
def binaryToDecimal(n): 
    return int(n,2) 

def decimalToBinary(n): 
    return bin(n).replace("0b","") 

In [48]:
def encrypt(msg):
    m = paddingOAEP(msg)
    print "Padded msg:" + m + "\n"
    a1 = ZZ(m)
    #print a1
    a = R(a1)
    print "Inteiro:" + a + "\n"
    cipher = a**e
    ct = mod(cipher,n)
    return ct

def decpryt(ct):
    dm = ct**d
    b = mod(dm, n)
    print "Depois de decifrar: " + b + "\n"
    m1 = ZZ(b)
    print "Inteiro depois de decifrar: " + m1
    m = m1.binary()
    print "Binário: " + m
    msg = unpad(m)
    return msg

In [47]:
from random import randint

msg = "Hello world"
#msg = randint(0,256)
#print msg
ct = encrypt(msg)
#print ct
m = decpryt(ct)
#print m

print msg == m

Padded msg:00100100001100101011011000110110001101111001000000111011101101111011100100110110001100100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100101100011111011000000101100110101111101011001100010011100110101011101000010111100000100101110100010000100000011001011010111100101100110000101111111100001100100010101101110001111110001011011001101000010110100010011100000001001000011101110010110011000110010110110010010010010111110000001111110111010110100101000111101001100110010011011001001011001110111001101011011110011101011111011011100011010001000111001111010101101101100111010100001101011010101101011011110011101111110011

/usr/lib/python2.7/dist-packages/sage/repl/ipython_kernel/__main__.py:6: DeprecationWarning: use 0o as octal prefix instead of 0
If you do not want this number to be interpreted as octal, remove the leading zeros.
See http://trac.sagemath.org/17413 for details.
